In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
sys.path.insert(0, "/Users/perceval/Development/metanno/")

In [46]:
app.state["buttons"]

[{'type': 'button', 'label': '⌫', 'color': 'white', 'key': 'Backspace'},
 {'type': 'spacer'},
 {'type': 'button',
  'label': 'a',
  'secondary': 'a',
  'color': 'rgb(255,200,206)',
  'key': 'a'},
 {'type': 'button',
  'label': 'b',
  'secondary': 'b',
  'color': 'rgb(210,236,247)',
  'key': 'b'},
 {'type': 'button',
  'label': 'c',
  'secondary': 'c',
  'color': 'rgb(211,242,206)',
  'key': 'c'},
 {'type': 'button',
  'label': 'd',
  'secondary': 'd',
  'color': 'rgb(242,242,206)',
  'key': 'd'},
 {'type': 'button',
  'label': 'e',
  'secondary': 'e',
  'color': 'rgb(231,210,247)',
  'key': 'e'},
 {'type': 'button',
  'label': 'f',
  'secondary': 'f',
  'color': 'rgb(252,215,216)',
  'key': 'f'},
 {'type': 'button',
  'label': 'g',
  'secondary': 'g',
  'color': 'rgb(251,243,219)',
  'key': 'g'},
 {'type': 'button',
  'label': 'h',
  'secondary': 'h',
  'color': 'rgb(250,231,212)',
  'key': 'h'},
 {'type': 'button',
  'label': 'i',
  'secondary': 'i',
  'color': 'rgb(250,212,229)',
  '

In [53]:
from metanno import *

colors = [
    "rgb(255,200,206)",
    "rgb(210,236,247)",
    "rgb(211,242,206)",
    "rgb(242,242,206)",
    "rgb(231,210,247)",
    "rgb(252,215,216)",
    "rgb(251,243,219)",
    "rgb(250,231,212)",
    "rgb(250,212,229)",
]

class SimpleApp(App):
    def __init__(self, dataset=None, labels=None):
        super().__init__()
        
        if dataset is None:
            self.state = {
                "doc_idx": 0,
                "dataset": [{"text": "", "annotations": []}],
                "mouse_selection": [],
                "highlighted": [],
                "styles": {},
                "buttons": [],
            }
            return
        
        if labels is None:
            labels = sorted(set(ann["label"] for doc in dataset for ann in doc["annotations"]))
        used_letters = set()
        buttons=[
            {"type": 'button', "label": "⌫", "color": 'white', "key": "Backspace"},
            {"type": 'spacer'},
        ]
        for label, color in zip(labels, colors):
            letter = next((l for l in label.lower() if l not in used_letters))
            used_letters.add(letter)
            buttons.append({"type": 'button', "label": label, "secondary": letter, "color": color, "key": letter})

        self.state = {
            "doc_idx": 0,
            "dataset": dataset if dataset is not None else [],
            "mouse_selection": [],
            "highlighted": [],
            "styles": {label: {"color": color, "alpha": 0.8} for label, color in zip(labels, colors)},
            "buttons": buttons,
        }
        
    def on_state_change(self, state, old_state):
        #logs.append(state["spans"] is not old_state["spans"])
        pass
    
    def select_editor_state(self, state, editor_id):
        if editor_id == "my-editor":
            if len(state["dataset"]):
                doc = state["dataset"][state["doc_idx"]]
            else:
                doc = {"text": "", "annotations": []}
            res = dict(
                text=doc["text"],
                spans=[{
                    "begin": span["begin"],
                    "end": span["end"],
                    "label": "Met",
                    "style": span["label"],
                    "id": i,
                    "highlighted": i in state["highlighted"],
                    "selected": False,#span["selected"],
                } for i, span in enumerate(doc["annotations"])],
                styles=state["styles"],
                mouse_selection=state["mouse_selection"],
                buttons=state["buttons"],
            )
            return res
        elif editor_id == "docs-table":
            return dict(
                rows=[{
                    "document": {"key": i, "text": doc["text"]},
                    "key": i,
                    "#entities": len(doc["annotations"]),
                    "highlighted": i == state["doc_idx"],
                } for i, doc in enumerate(state["dataset"])],
                rowKey="key",
                columns=[
                    {
                        "name": "document",
                        "type": "hyperlink",
                    },
                    {
                        "name": "#entities",
                        "readonly": True,
                        "type": "text",
                    },
                ],
                selectedRows=[],
            )
        elif editor_id == "mentions-table":
            text = state["dataset"][state["doc_idx"]]["text"]
            annotations = state["dataset"][state["doc_idx"]]["annotations"]
            res = dict(
                rows=[{
                    "key": i,
                    "highlighted": i in state["highlighted"],
                    "visible": True,
                    "mention": {
                        "text": text[span["begin"]:span["end"]],
                        "key": i,
                    },
                    "begin": span["begin"],
                    "end": span["end"],
                    "label": span["label"],
                } for i, span in enumerate(annotations)],
                rowKey="key",
                columns=[
                    {
                        "name": "begin",
                        "type": "text",
                    },
                    {
                        "name": "end",
                        "type": "text",
                    },
                    {
                        "name": "mention",
                        "type": "hyperlink",
                    },
                    {
                        "name": "label",
                        "type": "text",
                        "suggestions": [button["label"] for button in state["buttons"][2:]],
                    },
                ],
                selectedRows=[],
            )
            return res
        
    @produce
    def handle_mouse_select(self, editor_id, modkeys, spans):
        if "Shift" in modkeys:
            self.state["mouse_selection"].extend(spans)
        else:
            self.state["mouse_selection"] = spans
    
    @produce
    @frontend_only
    def handle_enter_span(self, editor_id, span_id, modkeys):
        self.state["highlighted"].append(span_id)
            
    @produce
    @frontend_only
    def handle_leave_span(self, editor_id, span_id, modkeys):
        if span_id in self.state["highlighted"]:
            self.state["highlighted"].remove(span_id)
    
    @produce
    def handle_key_press(self, editor_id, key, modkeys, spans):
        self.custom_command(key, spans)
        
    @produce
    def handle_button_press(self, editor_id, button_idx, selections):
        self.custom_command(self.state["buttons"][button_idx]["key"], selections)
        
    def custom_command(self, key, spans):
        def has_overlap(x, y):
            return not (x['end'] <= y['begin'] or y['end'] <= x['begin'])
        text = self.state["dataset"][self.state["doc_idx"]]["text"]
        if key == " ":
            first_selection = self.state["mouse_selection"][0]
            term = text[first_selection["begin"]:first_selection["end"]]
            next_occurence = self.state["text"].find(term, self.state["mouse_selection"][len(self.state["mouse_selection"])-1]["end"])
            if next_occurence > 0:
                self.state["mouse_selection"].append({"begin": next_occurence, "end": next_occurence + len(term)})
        if key == "Backspace":
            self.state["dataset"][self.state["doc_idx"]]["annotations"] = [
                span
                for span in self.state["dataset"][self.state["doc_idx"]]["annotations"]
                if not any(has_overlap(span, mouse_span) for mouse_span in spans)
            ]
            self.state.highlighted = []
        elif len(spans):
            new_spans = []
            for span in spans:
                for button in self.state["buttons"]:
                    if button["secondary"] == key:
                        new_spans.append({
                            "begin": span["begin"],
                            "end": span["end"],
                            "label": button["label"],
                        })
                        added_spans = True
            if not len(new_spans):
                return
            self.state["dataset"][self.state["doc_idx"]]["annotations"].extend(new_spans)
        self.state["mouse_selection"] = []
            
    @produce
    def handle_click_span(self, editor_id, span_id, modkeys):
        if "Shift" in modkeys:
            for span in self.state["spans"]:
                if span['id'] == span_id:
                    span['selected'] = not span['selected']
    
    @produce
    def handle_click_cell_content(self, editor_id, key):
        if editor_id == "docs-table":
            self.state["doc_idx"] = key
        else:
            self.scroll_to_span("my-editor", key)
            
    @produce
    def handle_cell_change(self, editor_id, row_idx, col, value):
        if col == "label":
            self.state["dataset"][self.state["doc_idx"]]["annotations"][row_idx]["label"] = value
    
# To update the app when we rerun the cell
try: app.set_class(SimpleApp)
except: pass

In [25]:
import base64

In [45]:
app = SimpleApp([
    {"text": "\nokok\n  \n", "annotations": []},
    {"text": "I became paranoid that the school of jellyfish was spying on me.", "annotations": []},
    {"text": "Nancy decided to make the porta-potty her home.", "annotations": []},
    {"text": "There is no better feeling than staring at a wall with closed eyes.", "annotations": []},
    {"text": "Of course, she loves her pink bunny slippers.", "annotations": []},
    {"text": "He set out for a short walk, but now all he could see were mangroves and water were for miles.", "annotations": []},
    {"text": "He was disappointed when he found the beach to be so sandy and the sun so sunny.", "annotations": []},
    {"text": "Patricia loves the sound of nails strongly pressed against the chalkboard.", "annotations": []},
    {"text": "The teenage boy was accused of breaking his arm simply to get out of the test.", "annotations": []},
    {"text": "They finished building the road they knew no one would ever use.", "annotations": []},
    {"text": "He found the end of the rainbow and was surprised at what he found there.", "annotations": []},
    {"text": "She had convinced her kids that any mushroom found on the ground would kill them if they touched it.", "annotations": []},
    {"text": "We have young kids who often walk into our room at night for various reasons including clowns in the closet.", "annotations": []},
    {"text": "Mom didn’t understand why no one else wanted a hot tub full of jello.", "annotations": []},
    {"text": "It was obvious she was hot, sweaty, and tired.", "annotations": []},
    {"text": "He quietly entered the museum as the super bowl started.", "annotations": []},
    {"text": "I've never seen a more beautiful brandy glass filled with wine.", "annotations": []},
    {"text": "He would only survive if he kept the fire going and he could hear thunder in the distance.", "annotations": []},
    {"text": "I’m working on a sweet potato farm.", "annotations": []},
    {"text": "100 years old is such a young age if you happen to be a bristlecone pine.", "annotations": []},
    {"text": "Art doesn't have to be intentional.", "annotations": []},
    {"text": "The door slammed on the watermelon.", "annotations": []},
    {"text": "You bite up because of your lower jaw.", "annotations": []},
    {"text": "Weather is not trivial - it's especially important when you're standing in it.", "annotations": []},
    {"text": "I had a friend in high school named Rick Shaw, but he was fairly useless as a mode of transport.", "annotations": []},
    {"text": "They called out her name time and again, but were met with nothing but silence.", "annotations": []},
    {"text": "The thunderous roar of the jet overhead confirmed her worst fears.", "annotations": []},
    {"text": "The tree fell unexpectedly short.", "annotations": []},
    {"text": "It was difficult for Mary to admit that most of her workout consisted of exercising poor judgment.", "annotations": []},
    {"text": "She was the type of girl that always burnt sugar to show she cared.", "annotations": []},
    {"text": "The hawk didn’t understand why the ground squirrels didn’t want to be his friend.", "annotations": []},
    {"text": "The swirled lollipop had issues with the pop rock candy.", "annotations": []},
    {"text": "Various sea birds are elegant, but nothing is as elegant as a gliding pelican.", "annotations": []},
    {"text": "She folded her handkerchief neatly.", "annotations": []},
    {"text": "Instead of a bachelorette party", "annotations": []},
    {"text": "She wanted a pet platypus but ended up getting a duck and a ferret instead.", "annotations": []},
    {"text": "The team members were hard to tell apart since they all wore their hair in a ponytail.", "annotations": []},
    {"text": "A song can make or ruin a person’s day if they let it get to them.", "annotations": []},
    {"text": "The green tea and avocado smoothie turned out exactly as would be expected.", "annotations": []},
    {"text": "She couldn't decide of the glass was half empty or half full so she drank it.", "annotations": []},
], ["a", "b", "c", "d", "e", "f", "g", "h", "i"])


In [14]:
app.state["dataset"][0]["annotations"].clear()

In [44]:
app.state["highlighted"].clear()

In [15]:
s._repr_mimebundle_()

{'text/plain': '<metanno.views.SpanEditor object at 0x7ff05a46d0a0>',
 'application/vnd.jupyter.annotator+json': {'version_major': 0,
  'version_minor': 0,
  'editor-type': 'span-editor',
  'editor-id': 'my-editor'}}

In [14]:
app.span_editor("my-editor")

In [69]:
app.table_editor("mentions-table")

In [40]:
app.table_editor("docs-table")

In [4]:
app.state

{'doc_idx': 0,
 'dataset': [{'text': 'Metanno Met anno', 'annotations': []},
  {'text': 'I became paranoid that the school of jellyfish was spying on me.',
   'annotations': []},
  {'text': 'Nancy decided to make the porta-potty her home.',
   'annotations': []},
  {'text': 'There is no better feeling than staring at a wall with closed eyes.',
   'annotations': []},
  {'text': 'Of course, she loves her pink bunny slippers.', 'annotations': []},
  {'text': 'He set out for a short walk, but now all he could see were mangroves and water were for miles.',
   'annotations': []},
  {'text': 'He was disappointed when he found the beach to be so sandy and the sun so sunny.',
   'annotations': []},
  {'text': 'Patricia loves the sound of nails strongly pressed against the chalkboard.',
   'annotations': []},
  {'text': 'The teenage boy was accused of breaking his arm simply to get out of the test.',
   'annotations': []},
  {'text': 'They finished building the road they knew no one would ever u

In [5]:
import pandas as pd

In [7]:
[dict(ann) for row in app.state["dataset"] for ann in row["annotations"]]

[{'begin': 0, 'end': 7, 'label': 'a'},
 {'begin': <metanno.immutable.AutoProxy at 0x7fbb567cbe50>,
  'end': <metanno.immutable.AutoProxy at 0x7fbb567cbee0>,
  'label': 'a'}]

In [11]:
#app.state["dataset"][0]["annotations"].clear()

In [20]:
app.state["dataset"]#[0]["annotations"]

[{'text': 'Metanno Met anno',
  'annotations': [{'begin': 0, 'end': 7, 'label': 'a'}]},
 {'text': 'I became paranoid that the school of jellyfish was spying on me.',
  'annotations': []},
 {'text': 'Nancy decided to make the porta-potty her home.',
  'annotations': []},
 {'text': 'There is no better feeling than staring at a wall with closed eyes.',
  'annotations': []},
 {'text': 'Of course, she loves her pink bunny slippers.', 'annotations': []},
 {'text': 'He set out for a short walk, but now all he could see were mangroves and water were for miles.',
  'annotations': []},
 {'text': 'He was disappointed when he found the beach to be so sandy and the sun so sunny.',
  'annotations': []},
 {'text': 'Patricia loves the sound of nails strongly pressed against the chalkboard.',
  'annotations': []},
 {'text': 'The teenage boy was accused of breaking his arm simply to get out of the test.',
  'annotations': []},
 {'text': 'They finished building the road they knew no one would ever use.',

In [7]:
pd.DataFrame([dict(ann) for row in app.state["dataset"] for ann in row["annotations"]])

""
0
1


In [21]:
pd.DataFrame([ann for row in app.state["dataset"] for ann in row["annotations"]])

,begin,end,label
0,0,7,a


In [22]:
pd.DataFrame(app.state["dataset"][0]["annotations"])

,begin,end,label
0,0,7,a
